In [38]:
# --- Importaciones ---
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import joblib

# --- Cargar dataset procesado ---
df = pd.read_csv(r"C:\Users\Luis Arbio\Documents\Hoteles\data\Processed\reservas_hoteles_fe.csv")

# --- Separar features y target ---
X = df.drop(['Booking_ID','booking_status'], axis=1)
y = df['booking_status']

# --- Dividir en train/test ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [39]:
# --- Definir modelos ---
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

results = {}

In [40]:
# --- Entrenar y evaluar métricas ---
for name, model in models.items():
    model.fit(X_train, y_train)
    
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    y_train_proba = model.predict_proba(X_train)[:,1]
    y_test_proba = model.predict_proba(X_test)[:,1]
    
    results[name] = {
        'train_accuracy': accuracy_score(y_train, y_train_pred),
        'test_accuracy': accuracy_score(y_test, y_test_pred),
        'train_f1': f1_score(y_train, y_train_pred),
        'test_f1': f1_score(y_test, y_test_pred),
        'train_roc': roc_auc_score(y_train, y_train_proba),
        'test_roc': roc_auc_score(y_test, y_test_proba)
    }

results_df = pd.DataFrame(results).T
print(results_df)

C:\Users\Luis Arbio\AppData\Roaming\Python\Python310\site-packages\xgboost\training.py:199: UserWarning: [20:41:50] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


                    train_accuracy  test_accuracy  train_f1   test_f1  \
LogisticRegression        0.808442       0.818194  0.684846  0.701786   
RandomForest              0.994142       0.903790  0.991032  0.846728   
XGBoost                   0.919056       0.895107  0.872261  0.833807   

                    train_roc  test_roc  
LogisticRegression   0.862632  0.871680  
RandomForest         0.999389  0.956724  
XGBoost              0.976616  0.954542  


In [41]:
# --- Optimización RandomForest con GridSearchCV ---
param_grid = {
    'n_estimators':[100,200],
    'max_depth':[None,10,20],
    'min_samples_split':[2,5]
}

grid = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='roc_auc')
grid.fit(X_train, y_train)
best_rf = grid.best_estimator_
print("Mejores parámetros RandomForest:", grid.best_params_)

Mejores parámetros RandomForest: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}


In [42]:
# --- Guardar modelo RandomForest optimizado ---
joblib.dump(best_rf, r"C:\Users\Luis Arbio\Documents\Hoteles\data\Processed\best_rf_model.pkl")
print("Modelo RandomForest guardado correctamente.")

Modelo RandomForest guardado correctamente.
